In [10]:
# Nota média do ENEM - Gráficos

In [11]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

# --- Base de dados ---
df = pd.read_csv('../dados/enem_2023_limpo.csv')

# Remove as categorias "Não Informado" e "Pública" para facilitar a visualização,
# já que esses valores são destinados ao treinamento dos modelos
df = df[~df['tipo_escola'].isin(['Não Informado', 'Pública'])].reset_index(drop=True)

# Calcula top 15 estados com maiores médias gerais
top15 = (
    df.groupby('sigla_estado')['nota_media'] #pegando media geral e as siglas e agrupando
    .mean()#Calcula media da nota_media
    .reset_index()#Esse comando devolve sigla_estado para ser uma coluna normal,
    #facilitando a manipulação e visualização.
    .sort_values('nota_media', ascending=False)#Ordena os estado do maior para menor valor
    #ascending=False indica ordem decrescente
    .head(15) #15 primeiros
)
top15_estados = top15['sigla_estado'].tolist() #pega so as siglas com uma conversão de lista python .tolist()

#  Cria tabela dinâmica (pivot) com a média geral por tipo de escola
pivot = df.pivot_table( #df.pivot grafico dinico
    index='sigla_estado',#cada linha para presenta um estado
    columns='tipo_escola',#tipo escola coluna
    values='nota_media',#Define os valores que vão aparecer
    aggfunc='mean'#função usda para agregar os dados é a média
)

#  Ordena as colunas (tipos de escola) pela média geral
media_geral_por_tipo_escola = df.groupby('tipo_escola')['nota_media'].mean().sort_values(ascending=False)
col_order = media_geral_por_tipo_escola.index.tolist()#df.groupby('tipo_escola')['nota_media'].mean(), calcula média geral
#por tipo de escola
#.sort_values ordena do maior para o menor valor
#media_geral_por_tipo_escola.index.tolist() -- Pega os nomes(Privada, Federal, Estadual) e converte
#para uma lista em python

#  Ordena os estados(Siglas) pela média geral
estado_order = df.groupby('sigla_estado')['nota_media'].mean().sort_values(ascending=False).index.tolist()

#  Reorganiza a tabela pivot
#linhas -> ordem dos estados(estado_order)
#colunas ->(col_order)
pivot = pivot.loc[estado_order, col_order]
pivot = pivot.apply(lambda x: x.fillna(x.mean()), axis=1)#preenche valores ausentes (NaN) com a média da própria linha.

#  Define limites da escala baseados na faixa de médias
zmin = 480   # limite mais baixo
zmax = 650   # limite mais alta

# Cria o heatmap
fig = px.imshow( #Cria uma imagem(heatmap) com base na tabela pivot
    pivot,
    text_auto='.1f',#Mostra o valor numérico dentro de cada célula com 1 casa decimal
    aspect='auto', #Ajusta a proporção automaticamente
    title='Média Geral por Tipo de Escola por Estado (ENEM 2023)', #Titulo em cima
    color_continuous_scale=[(0.0, "red"), (0.5, "yellow"), (1.0, "green")], #cores do grafico
                 #(VERMELHO, AMARELO, VERDE)
    zmin=zmin,#intervalo de valores para as cores minimo e max
    zmax=zmax
)

#  Aparência e formatação
fig.update_layout(
    title_x=0.5,#Centralizar
    xaxis_title='Tipo de Escola (ordenado pela média geral do tipo)',#Títulos de baixo
    yaxis_title='Estado (ordenado pela média geral)', #lado esquerdo
    coloraxis_colorbar=dict(title='Média Geral'),#Direito
    plot_bgcolor='rgba(0,0,0,0)',#RGB remove o fundo(Transparente)
    font=dict(size=10)#tamanho da fonte
)

#  Exibe
fig.show()

In [12]:
print('Média Geral por Tipo de Escola')
#Calculo da média geral por tipo de escola original
#media da escola, df.groupby Agrupa o DataFrame por tipo de escola
#dentro de cada grupo, acesa a coluna nota_media
media_escola = df.groupby('tipo_escola')['nota_media'].mean().reset_index().sort_values(by='nota_media', ascending=False)
#.mean, Calcula Média
# .reset_index() Tira o agrupamento do índice e transforma tipo_escola em uma coluna normal
#necessário para criar o grafico depois
#.sort... ordem maior para menor valor media geral

#Grafico de barras para media geral
fig = px.bar( #grafico de barras do pivot
    media_escola,#DataFrame usado para base dos dados
    x='tipo_escola',#baixo tipo escola
    y='nota_media',#esquerta nome media geral
    color='tipo_escola',# cor das escolas
    text='nota_media',#exibi o valor da media para cada barra
    title='Média Geral por Tipo de Escola', # titulo do grafico
    color_discrete_sequence=px.colors.qualitative.Set1 # paleta de cores
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
#texttemplate='%{text:.2f}', mostra o texto (a média) com duas casas decimais.
#textposition='outside', coloca o número fora da barra, logo acima.
fig.update_layout(
    xaxis_title='Tipo de Escola', #nomes do lados e baixo
    yaxis_title='Média Geral', #também
    title_x=0.5,#Centralizar titulo
    plot_bgcolor='rgba(0,0,0,0)', #fundo transparente
    showlegend=False # remove a legenda, porque as cores já representam claramente cada tipo de escola
)
fig.show() #Mostra o grafico interativo na tela(em jupyte Notebookm, VSCode, ou navegador)

Média Geral por Tipo de Escola


In [13]:
print('Percentual de alunos por Tipo de Escola')
# contagem de valores para a coluna original tipo_escola
contagem_pizza = df['tipo_escola'].value_counts().reset_index()
#Conta quantas vezes cada tipo de escola aparece no DataFrame
#.reset_index transforma o resultado em um dataFrame normal com duas colunas
contagem_pizza.columns = ['TIPO_ESCOLA', 'QUANTIDADE']
#Renomeia as colunas para ficar mais facil de usar

# Grafica Pizza
fig_pizza = px.pie( #pizza em pivot
    contagem_pizza,#DataFrame com os dados
    values='QUANTIDADE',#Tamanho de cada Fatia(Contagem de Alunos)
    names='TIPO_ESCOLA',#Nome exibido em cada fatia
    title='Distribuição Percentual de Alunos por Tipo de Escola', #Titulo do grafico
    color_discrete_sequence=px.colors.qualitative.Set2,#Cores Harmônicas
    hole=0.4 #Criar um buraco no meio
)
fig_pizza.update_traces(textinfo='percent+label', pull=[0.05]*len(contagem_pizza))
#Mostra, dentro ou fora das fatias percentual(%), categoria(Tipo escola)
#pull=[0.05]*len(contagem_pizza), Faz um pequeno Afastamento de todas as fatias (0.05 = 5%)
#len(contagem_pizza), Cria uma lista repetindo 0.05 para cada fatia do gráfico
fig_pizza.update_layout(title_x=0.5)
#Centraliza o titulo do gráfico no topo
fig_pizza.show()
#Mostra o gráfico interativo de pizza na tela(em Jupyter, VSCode ou navegador)

Percentual de alunos por Tipo de Escola


In [14]:
# --- Renda: Nota média por categoria (ordenado) ---
print('Renda — Nota média por categoria')

# Ordem desejada das categorias de renda
order_income = [
    'Muito Baixa', 'Baixa', 'Média-Baixa',
    'Média', 'Média-Alta', 'Alta', 'Muito Alta'
]

# Agrupamento e cálculo da média das notas
mean_renda = (
    df.groupby('categoria_renda', observed=False)['nota_media']
    .mean()#Calculo da Media
    .reindex(order_income)#Reorganiza as categorias na ordem definida acima(observed=False)
    .dropna()#Remove categorias que não possuem dados
    .reset_index()#Transforma o resultado de volta em DataFrame normal
)

# Gráfico de barras colorido pela média
fig = px.bar(
    mean_renda,
    x='categoria_renda', #Mostra faixa de renda
    y='nota_media', #Media das notas
    color='nota_media', #cor delas
    color_continuous_scale='Tealrose',  # escala bonita com contraste suave
    title='Média Geral por Categoria de Renda (ENEM 2023)', #Titulo
    labels={#Define rótulos personalizados dos eixos
        'categoria_renda': 'Categoria de Renda',
        'nota_media': 'Nota Média'
    },
    text_auto='.1f'  # mostra o valor numérico com 1 casa decimal
)

# Ajustes visuais
fig.update_layout(
    xaxis_title='Categoria de Renda (ordenada)',
    yaxis_title='Nota Média',
    coloraxis_colorbar_title='Nota Média',
    template='plotly_white',
    title_x=0.5
)

# Deixa o texto sempre preto para legibilidade
fig.update_traces(textfont_color='black')

fig.show()

Renda — Nota média por categoria
